In [1]:
# Import basic package to load the data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import os, sys
import warnings
warnings.filterwarnings('ignore')

import string
import re
import sqlite3

import nltk # Natural Language toolkit
from nltk.stem.wordnet import WordNetLemmatizer # Lemmatization
from nltk.stem.porter import PorterStemmer # Stemming
import re # Regular expression
from nltk.corpus import stopwords # STOPWORDS

from sklearn.feature_extraction.text import CountVectorizer # BOW
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer # TFIDF

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, classification_report , accuracy_score

In [59]:
con=sqlite3.connect('database.sqlite')
con

filtered_data=pd.read_sql_query("""select * from reviews where score !=3 limit 5000 """,con)
filtered_data.head()

In [60]:
filtered_data=pd.read_sql_query("""select * from reviews where score !=3""",con)
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [61]:
def partition(x):
    if x<3:
        return 0
    else:
        return 1

In [62]:
filtered_data['Score']=filtered_data['Score'].apply(partition)

In [63]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [64]:
filtered_data['Score'].value_counts()

1    443777
0     82037
Name: Score, dtype: int64

In [65]:
sorted_data=filtered_data.sort_values('ProductId')

In [66]:
sorted_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,1,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,1,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


In [67]:
sorted_data.shape

(525814, 10)

In [68]:
final=sorted_data.drop_duplicates(subset={'UserId', 'ProfileName','Text' })

In [69]:
final.shape

(363899, 10)

In [70]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

In [71]:
final.shape

(363897, 10)

!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_trf

# Text preprocessing -  tqdm
tqdm means "progress" in Arabic (taqadum, تقدّم) and is an abbreviation for "I love you so much" in Spanish (te quiero demasiado).
Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done!

In [72]:
from tqdm import tqdm
from bs4 import BeautifulSoup
stop_words = stopwords.words('english')

def decontracted(phrase):
    phrase = re.sub(r"isn't", "is not", phrase)
    phrase = re.sub(r"won't", "will not", phrase)
    return phrase

ppreviews=[]
for sentence in tqdm(final['Text']):
    sentence=re.sub(r'http\S','',sentence)
    sentence=BeautifulSoup(sentence,'lxml').getText()
    #sentence=decontracted(sentence)decontracted
    sentence=re.sub('[^a-zA-Z]+',' ',sentence)
    sentence=' '.join(e.lower() for e in sentence.lower().split() if e.lower() not in stop_words and len(e.lower())>1)
    ppreviews.append(sentence.strip())

100%|█████████████████████████████████████████████████████████████████████████| 363897/363897 [07:56<00:00, 764.02it/s]


In [73]:
ppreviews

['witty little book makes son laugh loud recite car driving along always sing refrain learned whales india drooping roses love new words book introduces silliness classic book willing bet son still able recite memory college',
 'grew reading sendak books watching really rosie movie incorporates love son loves however miss hard cover version paperbacks seem kind flimsy takes two hands keep pages open',
 'fun way children learn months year learn poems throughout school year like handmotions invent poem',
 'great little book read aloud nice rhythm well good repetition little ones like lines chicken soup rice child gets go months year go wonderful places like bombay nile eating well know get eat kids maurice sendak version ice skating treat roses heads long time even know came surprise came little witty book',
 'book poetry months year goes month cute little poem go along love book really fun way learn months poems creative author purpose writing book give children fun way learn months chi

# Feature extraction + modelling through Pipeline

In [74]:
# Train and test split
x_train,x_test,y_train,y_test=train_test_split(ppreviews,final['Score'],test_size=0.35,random_state=101)

In [79]:
len(x_train),len(x_test),len(y_train),len(y_test)

(236533, 127364, 236533, 127364)

# 1.Random Forest Model - only for limit 5000 dataset

In [ ]:
from sklearn.pipeline import Pipeline

classifier1=Pipeline([('tfidf',TfidfVectorizer()),('RandomForest',RandomForestClassifier(n_estimators=200))])
classifier1.fit(x_train,y_train)

In [31]:
y_pred_train_rf=classifier1.predict(x_train)
y_pred_test_rf=classifier1.predict(x_test)

In [32]:
print('Train accuracy',accuracy_score(y_train,y_pred_train_rf))
print('****'*5)
print('Test accuracy',accuracy_score(y_test,y_pred_test_rf))

Train accuracy 1.0
********************
Test accuracy 0.8458452722063037


In [33]:
print(classification_report(y_test,y_pred_test_rf))

              precision    recall  f1-score   support

           0       0.83      0.12      0.21       297
           1       0.85      1.00      0.91      1448

    accuracy                           0.85      1745
   macro avg       0.84      0.56      0.56      1745
weighted avg       0.84      0.85      0.79      1745



In [34]:
print(confusion_matrix(y_test,y_pred_test_rf))

[[  35  262]
 [   7 1441]]


In [35]:
from sklearn.model_selection import cross_val_score
train_accuracy = cross_val_score(classifier1,x_train,y_train ,cv=10)
test_accuracy = cross_val_score(classifier1,x_test,y_test, cv=10)
print(train_accuracy.mean())
print("="*50)
print(test_accuracy.mean())

0.857716049382716
0.8366830870279147


# 2. Naive Bayes Model - whole database

In [80]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
#MultinomialNB
classifier2 = Pipeline([("tfidf" , TfidfVectorizer()), ("MNB",MultinomialNB())])
classifier2.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('MNB', MultinomialNB())])

In [81]:
# Predict train and test dataset
y_pred_train_mnb = classifier2.predict(x_train)
y_pred_test_mnb = classifier2.predict(x_test)

In [82]:
print('Train accuracy',accuracy_score(y_train,y_pred_train_mnb))
print('****'*5)
print('Test accuracy',accuracy_score(y_test,y_pred_test_mnb))

Train accuracy 0.8590894293819467
********************
Test accuracy 0.8536635155931033


In [83]:
# BernoulliNB 
classifier3 = Pipeline([("tfidf" , TfidfVectorizer()), ("BNB",BernoulliNB())])
classifier3.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('BNB', BernoulliNB())])

In [84]:
# Predict train and test dataset
y_pred_train_bnb = classifier3.predict(x_train)
y_pred_test_bnb = classifier3.predict(x_test)

In [85]:
print('Train accuracy',accuracy_score(y_train,y_pred_train_bnb))
print('****'*5)
print('Test accuracy',accuracy_score(y_test,y_pred_test_bnb))

Train accuracy 0.8978408932368845
********************
Test accuracy 0.8889089538645143


In [ ]:
# Only with limit 5000 dataset
# GaussianNB - we need to convert message data to continous values so we need to convert tfidf toarray()
tfidf = TfidfVectorizer()
x=tfidf.fit_transform(ppreviews).toarray()
# Train and test split
x_train1,x_test1,y_train1,y_test1=train_test_split(x,final['Score'],test_size=0.35,random_state=101)
gnb = GaussianNB()
gnb.fit(x_train1,y_train1)

In [47]:
# Predict train and test dataset
y_pred_train_gnb = gnb.predict(x_train1)
y_pred_test_gnb = gnb.predict(x_test1)

In [48]:
print('Train accuracy',accuracy_score(y_train,y_pred_train_gnb))
print('****'*5)
print('Test accuracy',accuracy_score(y_test,y_pred_test_gnb))

Train accuracy 0.8895061728395062
********************
Test accuracy 0.7123209169054441


# 3.Support vector classifier model - only for limit 5000 dataset

In [42]:
from sklearn.svm import SVC
classifier4 = Pipeline([("tfidf" , TfidfVectorizer()), ("SVC",SVC(kernel='linear',gamma='auto'))])
classifier4.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('SVC', SVC(gamma='auto', kernel='linear'))])

In [43]:
# Predict train and test dataset
y_pred_train_svc = classifier4.predict(x_train)
y_pred_test_svc = classifier4.predict(x_test)

In [44]:
print('Train accuracy',accuracy_score(y_train,y_pred_train_svc))
print('****'*5)
print('Test accuracy',accuracy_score(y_test,y_pred_test_svc))

Train accuracy 0.9623456790123457
********************
Test accuracy 0.8796561604584527


# 4.Logistic Regression

In [87]:
from sklearn.linear_model import LogisticRegression
classifier5 = Pipeline([("tfidf" , TfidfVectorizer()) , ("log",LogisticRegression())])
classifier5.fit(x_train, y_train)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('log', LogisticRegression())])

In [88]:
# Predict train and test dataset
y_pred_train_log = classifier5.predict(x_train)
y_pred_test_log = classifier5.predict(x_test)

In [89]:
print('Train accuracy',accuracy_score(y_train,y_pred_train_log))
print('****'*5)
print('Test accuracy',accuracy_score(y_test,y_pred_test_log))

Train accuracy 0.9363640591376257
********************
Test accuracy 0.9275540969190667


# XGboost

In [91]:
#!pip install XGBoost

In [90]:
from xgboost import XGBClassifier
classifier6 = Pipeline([("tfidf" , TfidfVectorizer()) , ("XGB",XGBClassifier())])
classifier6.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('XGB',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [92]:
# Predict train and test dataset
y_pred_train_xgb = classifier6.predict(x_train)
y_pred_test_xgb = classifier6.predict(x_test)

In [93]:
print('Train accuracy',accuracy_score(y_train,y_pred_train_xgb))
print('****'*5)
print('Test accuracy',accuracy_score(y_test,y_pred_test_xgb))

Train accuracy 0.9205269455002051
********************
Test accuracy 0.9060959140730505


# Comparing models

  
1. Naive Bayes- MultinomialNB - 
    Train accuracy 0.8590894293819467
    ********************
    Test accuracy 0.8536635155931033
    
3. Naive Bayes - BernoulliNB - 
    Train accuracy 0.8978408932368845
    ********************
    Test accuracy 0.8889089538645143
    
4. Logistic Regression
    Train accuracy 0.9363640591376257
    ********************
    Test accuracy 0.9275540969190667    
7.XGBoost
    Train accuracy 0.9205269455002051
    ********************
    Test accuracy 0.9060959140730505

In [ ]:
So Xgboost,logistic regression are performing better than naive bayes model